In [2]:
import dspy
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [5]:
llm = dspy.OpenAI(model='gpt-3.5-turbo-0125', api_key=OPENAI_API_KEY, max_tokens=300)
dspy.settings.configure(lm=llm)

In [6]:
llm("hi how are you")

["Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?"]

#### Sentiment Classification 

In [7]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
classify(sentence=sentence).sentiment

'Positive'

In [12]:
short_story = dspy.Predict('sentence -> short_story')
short_story(sentence=sentence).short_story

"Sentence: it's a charming and often affecting journey.\nShort Story: Once upon a time, there was a charming and often affecting journey that captivated the hearts of all who embarked on it."

#### Summarization

In [13]:
# Example from the XSum dataset.
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

The 21-year-old player, who previously played for West Ham United and had loan spells in League One with Blackpool and Colchester United, has signed a contract with Barnsley, although the length of the contract has not been disclosed.


In [14]:
data = "./data/dogs.txt"
with open(data, "r") as f:
    document = f.read()

In [17]:
summarize = dspy.ChainOfThought('document -> tldr')
response = summarize(document=document)

print(response.tldr)

The Feline Financial Bank, founded by Sir Purrington, is a unique establishment exclusively for cats to secure their belongings in an elegant and cat-friendly environment, offering services like savings accounts and high-security vaults, symbolizing feline independence and sophistication.


#### Classification

In [18]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""
    
    sentence = dspy.InputField()
    sentiment = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='Fear'
)